In [1]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
spark_version = 'spark-3.1.2'
# spark_version = 'spark-3.<enter version>'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
#!wget -q https://downloads.apache.org/spark/spark-3.0.1/spark-3.0.1-bin-hadoop3.2.tgz
!wget -q https://archive.apache.org/dist/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15

In [2]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("sparkFunctions").getOrCreate()

In [3]:
from pyspark import SparkFiles
url ="https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Watches_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Watches_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)

# Show DataFrame
df.show(truncate = False)

+-----------+-----------+--------------+----------+--------------+----------------------------------------------------------------------------------------------------+----------------+-----------+-------------+-----------+----+-----------------+----------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------+
|marketplace|customer_id|review_id     |product_id|product_parent|product_title                                     

In [4]:
# Count the # of watches in dataset 2
df.count()

960872

In [5]:
from pyspark.sql.functions import to_date
# Transform DataFrame
review_id_table = df.select(['review_id', 'customer_id', 'product_id', 'product_parent', to_date("review_date","yyyy-MM-dd").alias("review_date")])
review_id_table.show(truncate = False)

+--------------+-----------+----------+--------------+-----------+
|review_id     |customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R3O9SGZBVQBV76|3653882    |B00FALQ1ZC|937001370     |2015-08-31 |
|RKH8BNC3L5DLF |14661224   |B00D3RGO20|484010722     |2015-08-31 |
|R2HLE8WKZSU3NL|27324930   |B00DKYC7TK|361166390     |2015-08-31 |
|R31U3UH5AZ42LL|7211452    |B000EQS1JW|958035625     |2015-08-31 |
|R2SV659OUJ945Y|12733322   |B00A6GFD7S|765328221     |2015-08-31 |
|RA51CP8TR5A2L |6576411    |B00EYSOSE8|230493695     |2015-08-31 |
|RB2Q7DLDN6TH6 |11811565   |B00WM0QA3M|549298279     |2015-08-31 |
|R2RHFJV0UYBK3Y|49401598   |B00A4EYBR0|844009113     |2015-08-31 |
|R2Z6JOQ94LFHEP|45925069   |B00MAMPGGE|263720892     |2015-08-31 |
|RX27XIIWY5JPB |44751341   |B004LBPB7Q|124278407     |2015-08-31 |
|R15C7QEZT0LGZN|9962330    |B00KGTVGKS|28017857      |2015-08-31 |
|R361XSS37V0NCZ|16097204   |B0039UT5OU|685450910     |2015-08-

In [6]:
# This table will contain only unique values
product_df = df.select(['product_id', 'product_title']).drop_duplicates()
product_df.show(truncate = False)

+----------+---------------------------------------------------------------------------------------------------------+
|product_id|product_title                                                                                            |
+----------+---------------------------------------------------------------------------------------------------------+
|B00EVX7V1I|Game Time Women's NFL Frost Series Watch                                                                 |
|B009S4DODY|XOXO Women's XO5571 Silver-Tone Watch with Mesh Strap                                                    |
|B00LBKXQRW|Anne Klein Women's AK/1814WTGB Glitter Accented Gold-Tone and White Ceramic Bracelet Watch               |
|B0009P679Y|Invicta Men's 9938 Pro Diver Collection Automatic Diver Watch                                            |
|B00DHF30RU|M&c Women's | Classy Crystal Embedded Bangle Look Watch with Self-Adjustable Bracelet | FC0365           |
|B00NIDA43Y|GuTe Classic Skeleton Mechanical Wri

In [7]:
#  Customer table for first data set
customer_df = df.groupby('customer_id').count().withColumnRenamed('count', 'customer_count')
customer_df.show(truncate = False)

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|1567510    |1             |
|19502021   |1             |
|12819130   |1             |
|35329257   |2             |
|108460     |1             |
|5453476    |1             |
|29913055   |1             |
|30717305   |1             |
|1570030    |1             |
|19032020   |1             |
|44178035   |1             |
|26079415   |2             |
|14230926   |1             |
|43478048   |2             |
|43694941   |1             |
|12318815   |3             |
|13731855   |1             |
|740134     |1             |
|41956754   |1             |
|20324070   |3             |
+-----------+--------------+
only showing top 20 rows



In [8]:
# vine table
vine_df = df.select(['review_id', 'star_rating', 'helpful_votes', 'product_parent', 'total_votes', 'vine'])
vine_df.show(truncate = False)

+--------------+-----------+-------------+--------------+-----------+----+
|review_id     |star_rating|helpful_votes|product_parent|total_votes|vine|
+--------------+-----------+-------------+--------------+-----------+----+
|R3O9SGZBVQBV76|5          |0            |937001370     |0          |N   |
|RKH8BNC3L5DLF |5          |0            |484010722     |0          |N   |
|R2HLE8WKZSU3NL|2          |1            |361166390     |1          |N   |
|R31U3UH5AZ42LL|5          |0            |958035625     |0          |N   |
|R2SV659OUJ945Y|4          |0            |765328221     |0          |N   |
|RA51CP8TR5A2L |5          |0            |230493695     |0          |N   |
|RB2Q7DLDN6TH6 |5          |1            |549298279     |1          |N   |
|R2RHFJV0UYBK3Y|1          |1            |844009113     |5          |N   |
|R2Z6JOQ94LFHEP|5          |1            |263720892     |2          |N   |
|RX27XIIWY5JPB |4          |0            |124278407     |0          |N   |
|R15C7QEZT0LGZN|4        